## 44단계: 매개변수를 모아두는 계층층

> .

### 44.1 Parameter 클래스 구현

In [1]:
# dezero/core.py
from dezero import Variable

class Parameter(Variable):
    pass

In [2]:
import numpy as np

x = Variable(np.array(1.0))
p = Parameter(np.array(2.0))
y = x * p

print(isinstance(p, Parameter))
print(isinstance(x, Parameter))
print(isinstance(y, Parameter))

True
False
False


### 44.2 Layer 클래스 구현

In [3]:
# dezero/layers.py
from dezero.core import Parameter

class Layer:
    def __init__(self):
        self._params = set()
    
    def __setattr__(self, name, value):
        if isinstance(value, Parameter):
            self._params.add(name)
        super().__setattr__(name, value)

In [4]:
from dezero import Variable

layer = Layer()

layer.p1 = Parameter(np.array(1))
layer.p2 = Parameter(np.array(2))
layer.p3 = Variable(np.array(3))
layer.p4 = 'test'

print(layer._params)
print('------------')

for name in layer._params:
    print(name, layer.__dict__[name])

{'p2', 'p1'}
------------
p2 Variable(2)
p1 Variable(1)


In [ ]:
# dezero/layers.py

# Layer 클래스 추가 구현
import weakref

class Layer:
    ...

    def __call__(self, *inputs):
        outputs = self.forward(*inputs)
        if not isinstance(outputs, tuple):
            outputs = (outputs,)
        self.inputs = [weakref.proxy(input) for x in inputs]
        self.outputs = [weakref.proxy(input) for y in outputs]
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, inputs):
        raise NotImplementedError()
    
    def params(self):
        for name in self._params:
            yield self.__dict__[name]
        
    def cleargrads(self):
        for param in self.params():
            param.cleargrad()

### 44.3 Linear 클래스 구현

In [6]:
# dezero/layers.py

import numpy as np
import dezero.functions as F
from dezero.core import Parameter

class Linear(Layer):
    def __init__(self, out_size, nobias=False, dtype=np.float32, in_size=None):
        super().__init__()
        self.in_size = in_size
        self.out_size = out_size
        self.dtype = dtype

        self.W = Parameter(None, name='W')
        if self.in_size is not None:
            self._init_W()

        if nobias:
            self.b = None
        else:
            self.b = Parameter(np.zeros(out_size, dtype=dtype), name='b')

    def _init_W(self):
        I, O = self.in_size, self.out_size
        W_data = np.random.randn(I, O).astype(self.dtype) * np.sqrt(1 / I)
        self.W.data = W_data

    def forward(self, x):
        if self.W.data is None:
            self.in_size = x.shape[1]
            self._init_W()

        y = F.linear(x, self.W, self.b)
        return y

### 44.4 Layer를 이용한 신경망 구현

In [7]:
import numpy as np
import dezero.functions as F
import dezero.layers as L


np.random.seed(0)
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100, 1)

l1 = L.Linear(10)
l2 = L.Linear(1)


def predict(x):
    y = l1(x)
    y = F.sigmoid(y)
    y = l2(y)
    return y


lr = 0.2
iters = 10000

for i in range(iters):
    y_pred = predict(x)
    loss = F.mean_squared_error(y, y_pred)

    l1.cleargrads()
    l2.cleargrads()
    loss.backward()

    for l in [l1, l2]:
        for p in l.params():
            p.data -= lr * p.grad.data
    if i % 1000 == 0:
        print(loss)

Variable(0.8165178479901415)
Variable(0.2499028014603371)
Variable(0.24609874026436834)
Variable(0.23721586110833612)
Variable(0.20793217994822147)
Variable(0.12311919860580518)
Variable(0.07888168390348675)
Variable(0.07656075297857305)
Variable(0.07643364647799142)
Variable(0.0761937449484299)
